# Convolutional autoencoder 

In [1]:
from tensorflow.keras.datasets import mnist
import numpy as np

## Load data

In [2]:
#load the data
(xtr,_),(xtest,ytest) = mnist.load_data()

xtr = xtr.reshape((-1,28,28,1)).astype('float32')/255
xtest = xtest.reshape((-1,28,28,1)).astype('float32')/255

## Create model

### Encoder

In [3]:
#Layers import 
from tensorflow.keras.layers import Dense, Flatten,Reshape, Conv2D,Conv2DTranspose, Input
from tensorflow.keras.backend import int_shape
from tensorflow.keras.models import Model

latentSize = 16

In [4]:
inputs = Input(shape=xtr[0].shape, name='Input_layer' )

x = Conv2D(kernel_size=3,filters=16,strides=1,padding='same')(inputs)
x = Conv2D(kernel_size=3,filters=32,strides=1,padding='same')(x)
x = Conv2D(kernel_size=3,filters=64,strides=1,padding='same')(x)

shp = int_shape(x)

x = Flatten()(x)

out = Dense(latentSize,name="latent_space")(x)

encoder = Model(inputs,out, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        4640      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
latent_space (Dense)         (None, 16)                802832    
Total params: 826,128
Trainable params: 826,128
Non-trainable params: 0
_____________________________________________________

### Decoder

In [5]:
latentSpace = Input(shape=latentSize,name="latent_space_in" )

x = Dense(shp[1]*shp[2]*shp[3])(latentSpace)
x = Reshape((shp[1],shp[2],shp[3]))(x)

x = Conv2DTranspose(kernel_size=3,filters=64,strides=1,padding='same')(x)
x = Conv2DTranspose(kernel_size=3,filters=32,strides=1,padding='same')(x)
x = Conv2DTranspose(kernel_size=3,filters=16,strides=1,padding='same')(x)

outputs = Conv2DTranspose(kernel_size=3,filters=1,strides=1,padding='same')(x)


decoder = Model(latentSpace,outputs,name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
latent_space_in (InputLayer) [(None, 16)]              0         
_________________________________________________________________
dense (Dense)                (None, 50176)             852992    
_________________________________________________________________
reshape (Reshape)            (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 28, 28, 64)        36928     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 28, 28, 16)        4624      
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 28, 28, 1)         145 

### Autoencoder

In [6]:
output = decoder(encoder(inputs))
autoencoder = Model(inputs,output,name='autoencoder')
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     [(None, 28, 28, 1)]       0         
_________________________________________________________________
encoder (Model)              (None, 16)                826128    
_________________________________________________________________
decoder (Model)              (None, 28, 28, 1)         913153    
Total params: 1,739,281
Trainable params: 1,739,281
Non-trainable params: 0
_________________________________________________________________


In [7]:
autoencoder.compile(loss='mse',optimizer='adam',metric=['accuracy'])

In [8]:
autoencoder.fit(x=xtr,y=xtr,validation_data=[xtest,xtest],epochs=20,batch_size=1024)

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 31s 510us/sample - loss: 0.0441 - val_loss: 0.0286
Epoch 2/20
60000/60000 [==============================] - 25s 423us/sample - loss: 0.0283 - val_loss: 0.0273
Epoch 3/20
60000/60000 [==============================] - 25s 423us/sample - loss: 0.0276 - val_loss: 0.0271
Epoch 4/20
60000/60000 [==============================] - 25s 423us/sample - loss: 0.0276 - val_loss: 0.0271
Epoch 5/20
60000/60000 [==============================] - 25s 421us/sample - loss: 0.0276 - val_loss: 0.0271
Epoch 6/20
60000/60000 [==============================] - 25s 423us/sample - loss: 0.0275 - val_loss: 0.0271
Epoch 7/20
60000/60000 [==============================] - 26s 427us/sample - loss: 0.0275 - val_loss: 0.0270
Epoch 8/20
60000/60000 [==============================] - 26s 426us/sample - loss: 0.0274 - val_loss: 0.0270
Epoch 9/20
60000/60000 [==============================] - 26s 430us/sample - l